# WebScraping: Notas completas

In [2]:
#Importes necesarios
import pymysql as sql
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
#Conectar a la base de datos en MySQL local
conn = sql.connect(
    host="localhost",
    port=3306,
    user="root",
    passwd="Password1234!",
    db="siglonuevo")
cursor = conn.cursor()

In [14]:
#Acceder, seleccionar y variabilizar el url para scraping
with conn.cursor() as cursor: 
    i=1
    while i<8:
        #Seleccionar URL de base de datos
        cursor.execute("SELECT `enlace` FROM `siglodb` WHERE id=(%s)",(i))
        url_obtenido = cursor.fetchone()
        #Quitar paréntesis y comillas de URL resultante
        result1 = str(url_obtenido).replace("',)", "")
        url_obtenido_texto = str(result1).replace("('","")
        
        #Seleccionar información
        html = urlopen(url_obtenido_texto)
        soup = BeautifulSoup(html, 'lxml')
        seccion = soup.find('a',{'class': 'news-point'})
        titular = soup.find('h1')
        balazo = soup.find('h2')
        autor = soup.find('span',{'class': 'news-author__name'})
        primer_parrafo = soup.find('p',{'class': 'news-first'})
        cuerpo = soup.find('div',{'class': 'news-text-contain'})
        #Pendiente borrar redes sociales
        if seccion is None:
            pass
        else:
            seccion_strip = seccion.get_text().strip()
            cursor.execute("UPDATE `siglodb` SET `seccion` = (%s) WHERE `id` = (%s)", (seccion_strip,i))
        if titular is None:
            pass
        else:
            titular_strip = titular.get_text().strip()
            cursor.execute("UPDATE `siglodb` SET `titular` = (%s) WHERE `id` = (%s)", (titular_strip,i))
        if balazo is None:
            pass
        else:
            balazo_strip = balazo.get_text().strip()
            cursor.execute("UPDATE `siglodb` SET `balazo` = (%s) WHERE `id` = (%s)", (balazo_strip,i))
        if autor is None:
            pass
        else:
            autor_strip = autor.get_text().strip()
            cursor.execute("UPDATE `siglodb` SET `autor` = (%s) WHERE `id` = (%s)", (autor_strip,i))
        if primer_parrafo is None:
            pass
        else:
            primer_parrafo_strip = primer_parrafo.get_text().strip()
            cursor.execute("UPDATE `siglodb` SET `primer_parrafo` = (%s) WHERE `id` = (%s)", (primer_parrafo_strip,i))
        i += 1
        conn.commit()